In [ ]:
# Imports
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.arima.model import ARIMA
import seaborn as sns
import numpy as np

# Load dataset
df = pd.read_csv('C:/Users/LuisEduardo/OneDrive/Data analyst Portfolio/Natural Gas Forecast/daily_csv.csv')
#df = pd.read_csv('C:/Users/LuisEduardo/OneDrive/Data analyst Portfolio/Natural Gas Forecast/Gas_Prices_SoCal.csv')


# Display initial data
df.tail()

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

In [ ]:
# Replace infinite values
df['Price'].replace([np.inf, -np.inf], np.nan, inplace=True)

# Dealing NaN values
df['Price'].fillna(df['Price'].mean(), inplace = True)
#df.dropna(subset=['Price'], inplace=True)

In [ ]:
# Plot histogram of price distribution
sns.set_style('darkgrid')
sns.set_color_codes(palette='dark')

f, ax = plt.subplots(figsize=(9, 5))

sns.histplot(df['Price'], color="m", kde=True, ax=ax)
ax.set(title="Histogram for SalePrice", xlabel='Price', ylabel='Frequency')

plt.show()

In [ ]:
# Convert 'Dates' column to datetime for better visualization
df.rename(columns={'TRADE_DATE': 'Date'}, inplace=True)

In [ ]:
# Convert the 'DATE' column to datetime with the correct format
df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y', dayfirst=True)

In [ ]:
# Natural Gas prices over time
plt.figure(figsize=(14, 6))

sns.lineplot(x='Date', y='Price', data=df)

plt.title('Natural Gas Prices Over Time - Daily')
plt.xlabel('Date')
plt.ylabel('Price')
plt.grid(True)

plt.xticks(rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
# Detect outliers
def detect_outliers(df, n, features):

    outlier_indices = []

    for col in features:

        Q1 = np.percentile(df[col], 25)
        Q3 = np.percentile(df[col], 75)

        IQR = Q3 - Q1
        print('\n',col)

        outlier_step = 1.5 * IQR

        outlier_list_col = df[(df[col] < Q1 - outlier_step) | (df[col] > Q3 + outlier_step)].index
        print(outlier_list_col)
        outlier_indices.extend(outlier_list_col)
    
    return outlier_indices

In [ ]:
# Remove outliers
lof = ['Price']
data = df[['Date','Price']]
Outliers_to_drop = detect_outliers(data, 1, lof)

In [ ]:
data.drop(Outliers_to_drop,inplace=True)
data.reset_index(drop=True, inplace=True)

In [ ]:
y = df['Price']
size = df.size
print(str(y[Outliers_to_drop].size) + "/" + str(size) + " data points droped.")
print("Turkey Method, " + str(round(100-((y[Outliers_to_drop].size/size)*100),2)) +  " % remaining data points from the Dataset.")

In [ ]:
# # Natural Gas prices over time (without outliers)
plt.figure(figsize=(14, 6))

sns.lineplot(x='Date', y='Price', data=df)
sns.lineplot(x='Date', y='Price', data=data)

plt.title('Natural Gas Prices Over Time - removing outliers')
plt.xlabel('Date')
plt.ylabel('Price')
plt.grid(True)

plt.gca().xaxis.set_major_locator(plt.matplotlib.dates.YearLocator())  # Muestra cada año

plt.xticks(rotation=45)

plt.tight_layout()
plt.show()

### Taking the monthly average, since the purpose is to forecast future monthly prices 

In [ ]:
# Create a new column with the format 'YYYY-MM'
data['YEAR_MONTH'] = data['Date'].dt.to_period('M')

# Group by the new column 'YEAR_MONTH' and calculate the average price per month
monthly_avg_price = data.groupby('YEAR_MONTH')['Price'].agg(['mean','std']).reset_index()
monthly_avg_price['cv'] = (monthly_avg_price['std']/monthly_avg_price['mean'])

# Convert 'YEAR_MONTH' from Period to string for final presentation
monthly_avg_price['YEAR_MONTH'] = monthly_avg_price['YEAR_MONTH'].astype(str)

# Rename columns if necessary
monthly_avg_price.columns = ['Month_Year', 'Average_Price', 'std', 'cv']
monthly_avg_price['Month_Year'] = pd.to_datetime(monthly_avg_price['Month_Year'])

In [ ]:
f, axes = plt.subplots(1, figsize=(15,5))

sns.lineplot(x="Month_Year", y="std",label ='Std',data=monthly_avg_price)
sns.lineplot(x="Month_Year", y="cv",label ='CV',data=monthly_avg_price)
#sns.lineplot(x="Month_Year", y="Average_Price",label ='mean',data=monthly_avg_price)
plt.ylabel('Measures')

plt.title("Natural Gas - The Standard Deviation Monthly Prices")

plt.show()

The time series has a non-constant standard deviation over time.

In [ ]:
# Decompose the time series to study its components
decomposition = seasonal_decompose(monthly_avg_price['Average_Price'], period=12)  # Assuming yearly seasonality with monthly data

# Plot decomposition
fig, axes = plt.subplots(4, 1, figsize=(14, 10))

labels = ['Original', 'Trend', 'Seasonal', 'Residual']
components = [monthly_avg_price['Average_Price'], decomposition.trend, decomposition.seasonal, decomposition.resid]

for i, ax in enumerate(axes):
    ax.plot(monthly_avg_price['Month_Year'], components[i], label=labels[i])
    ax.legend(loc='upper left')
    ax.set_title(f"{labels[i]} Component")
    ax.grid(True)

plt.tight_layout()
plt.show()

- **Original Component:** <br>
This is the original time series as observed. <br>
It shows variations and patterns over time from 1996 to 2024. <br>
We note significant peaks around 2000, 2008, and 2020. <br>

- **Trend Component:** <br>
This component shows the long-term trend of the time series. <br>
We see a steady increase from 1996 to about 2007, followed by a decrease until 2012. <br>
After 2012, the trend is quite variable with a peak around 2020 followed by a decline. <br>
This trend captures the large long-term variations without the seasonal fluctuations and noise. <br>

- **Seasonal Component:** <br>
This component shows recurring seasonal patterns that repeat in a regular cycle. <br>
In this case, the cycles appear to be annual due to the regularity and constant repetition of the pattern. <br>
Seasonal fluctuations are small compared to trend changes and residuals, indicating that seasonality has a minor but consistent impact. <br>

- **Residual Component:** <br>
Residuals represent random noise or any variation that cannot be explained by trend or seasonality. <br>
We observe some notable peaks, especially around 2000, 2008 and 2020, suggesting irregular events or anomalies in those periods. <br>

In [ ]:
# Test stationarity of the series
def is_stationary(timeseries):
    """Check stationarity using the Augmented Dickey-Fuller test."""
    dftest = adfuller(timeseries, autolag='AIC')
    return dftest[1] <= 0.05
# Check original and differenced series
is_stationary_original = is_stationary(monthly_avg_price['Average_Price'])

#We then create a differenced series to transform a potentially non-stationary series into a stationary one.
monthly_avg_price.loc[:, 'First Difference'] = monthly_avg_price['Average_Price'].diff()   #calculates the difference between consecutive values in the Price series. This helps in stabilizing the mean of the time series by removing changes in the level of the series, thus potentially making it stationary.
#monthly_avg_price.loc[:, 'Seasonal Difference'] = monthly_avg_price['Average_Price'].diff(periods=12) 

is_stationary_first_diff = is_stationary(monthly_avg_price['First Difference'].dropna())
#is_stationary_season_diff = is_stationary(monthly_avg_price['Seasonal Difference'].dropna())
is_stationary_original, is_stationary_first_diff


The Augmented Dickey-Fuller test indicates that the original saerie is not stationary. By differencing the series, it becomes stationary.

In [ ]:
# Plot ACF and PACF for insights into AR and MA terms
fig, axes = plt.subplots(1, 2, figsize=(14, 4))

# Plot ACF
plot_acf(monthly_avg_price['First Difference'].dropna(), lags=24, ax=axes[0])
axes[0].set_title('ACF for First Differenced Series')

# Plot PACF with method='ywm'
plot_pacf(monthly_avg_price['First Difference'].dropna(), lags=24, ax=axes[1], method='ywm')
axes[1].set_title('PACF for First Differenced Series')

plt.tight_layout()

plt.show()

### Analysis of ACF and PACF Charts:


**Selection of \( p \) (Autoregressive Order)**:
- The PACF plot of the differenced series shows a significant spike at the first lag.
- The significance at the first lag suggests that an AR(1) model is appropriate, implying \( p = 1 \).
- No significant correlations are observed at subsequent lags, so a higher order is not necessary.

**Selection of \( d \) (Differencing Order)**:
- The time series exhibits a non-stationary trend, justifying the need for differencing.
- A first-order differencing (\( d = 1 \)) is applied to achieve stationarity, confirmed by the stabilization of the mean and variance in the differenced series.

**Selection of \( q \) (Moving Average Order)**:
- The ACF plot of the differenced series shows that, after the first lag, all autocorrelations are within the confidence interval.
- The lack of significant autocorrelations beyond the first lag indicates that an additional moving average component is unnecessary.
- Therefore, \( q = 0 \) is selected to maintain model simplicity.

### Final Model Configuration
Based on the analysis of ACF and PACF, the final configuration of the SARIMAX model is:
- **Order**: (1, 1, 0)
- **Seasonal Order**: (1, 1, 1, 12)



In [ ]:
# Fit SARIMA model

model = SARIMAX(monthly_avg_price['Average_Price'], order=(1,1,0), seasonal_order=(1,1,1,12))
#model = ARIMA(monthly_avg_price['Average_Price'], order=(1,1,0))
results = model.fit()
results.summary()


In [ ]:
# Compare actual and fitted values
monthly_avg_price['Fitted'] = results.fittedvalues

plt.figure(figsize=(14, 6))
plt.plot(monthly_avg_price['Month_Year'], monthly_avg_price['Average_Price'], label='Actual', color='blue')
plt.plot(monthly_avg_price['Month_Year'], monthly_avg_price['Fitted'], label='Fitted', color='red', linestyle='--')
plt.title('Actual vs Fitted Natural Gas Prices')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# Forecast for the next three years
forecast = results.get_forecast(steps=12)
forecast_index = pd.date_range(monthly_avg_price['Month_Year'].iloc[-1] + pd.Timedelta(days=1), periods=12, freq='M')
forecast_series = pd.Series(forecast.predicted_mean.values, index=forecast_index)
print(forecast_series)

plt.figure(figsize=(14, 6))
plt.plot(monthly_avg_price['Month_Year'], monthly_avg_price['Average_Price'], label='Historical', color='blue')
plt.plot(forecast_index, forecast_series, label='Forecast', color='green', linestyle='--')
#plt.fill_between(forecast_index, forecast.conf_int()['lower Prices'], forecast.conf_int()['upper Prices'], color='green', alpha=0.1)
plt.title('Natural Gas Prices Forecast')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

### Cross Validation

In [ ]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error

tscv = TimeSeriesSplit(n_splits=5)

In [ ]:
# Define a function to train the model and calculate the MSE
def train_sarimax(train, test):
    model = SARIMAX(train, order=(1,1,0), seasonal_order=(1,1,1,12))
    results = model.fit(disp=False)
    predictions = results.forecast(steps=len(test))
    mse = mean_squared_error(test, predictions)
    return mse


mse_scores = []

for train_index, test_index in tscv.split(monthly_avg_price):
    train, test = monthly_avg_price['Average_Price'].iloc[train_index], monthly_avg_price['Average_Price'].iloc[test_index]
    mse = train_sarimax(train, test)
    mse_scores.append(mse)

    if len(train) < 2 or len(test) < 1:
        continue  # Skip this fold if not enough data    

print("MSE Scores for each fold: ", mse_scores)
print("Average MSE: ", np.mean(mse_scores))

#### **Interpretation**

**Variability in MSE Scores:**
- There is considerable variability in the MSE values between the different folds. The first fold has a significantly higher MSE (11.95) compared to the fourth fold (0.28). This may indicate that the model has difficulty predicting certain parts of the data.<br>
- Differences in MSE values may be due to seasonal changes, trends, or specific anomalous events in the data for the different folds.

**Average MSE:**
- The average MSE of 4.437 indicates the overall mean square error of the model across all folds. A lower MSE generally indicates better model performance. However, since MSE values vary widely between folds, it is important to consider the distribution of errors as well.

**Model Evaluation:**
- Overall Performance: Although the average MSE is not very high, the high variability suggests that the model may be performing well in some periods and not as well in others. This could be an indication that the SARIMAX model may not be fully capturing all features of the time series, such as more complex seasonal variations or abrupt changes in the series.

### Review and Adjust Model Hyperparameters

In [ ]:
from sklearn.model_selection import ParameterGrid

# Define hyperparameter ranges for searching
p = range(0, 3)
d = range(0, 2)
q = range(0, 3)
P = range(0, 3)
D = range(0, 2)
Q = range(0, 3)
m = [12]

# Create a hyperparameter grid
param_grid = ParameterGrid({
    'order': [(x[0], x[1], x[2]) for x in zip(p, d, q)],
    'seasonal_order': [(x[0], x[1], x[2], x[3]) for x in zip(P, D, Q, m)]
})

best_score = float('inf')
best_params = None

for params in param_grid:
    model = SARIMAX(monthly_avg_price['Average_Price'], order=params['order'], seasonal_order=params['seasonal_order'])
    results = model.fit(disp=False)
    mse = mean_squared_error(monthly_avg_price['Average_Price'], results.fittedvalues)
    if mse < best_score:
        best_score = mse
        best_params = params

print("Best Params:", best_params)

The optimal configuration found indicates that the model does not need to capture seasonal components (P, D, Q) and that a simple ARIMA (1, 1, 1) model is sufficient. This suggests that the model is probably not capturing well the seasonal patterns and variability of the data.

# Another time series implementation (Prophet)

In [ ]:
from prophet import Prophet

In [ ]:
# Selecting a period of time if necesary
data_p = data.copy()
#data_p = data[data['Date'] >= '2000-01-01'].copy()
#data_p.reset_index(drop=True, inplace=True)



In [ ]:
# Remove outliers
#Outliers_to_drop = detect_outliers(data_p[['Date','Price']], 1, ['Price'])
#data_p.drop(Outliers_to_drop,inplace=True)
#data_p.reset_index(drop=True, inplace=True)

In [ ]:
prior_scale=0.02
prices_prophet = data_p[['Date','Price']]

In [ ]:
m = Prophet(changepoint_prior_scale=prior_scale, interval_width=0.95)
prices_prophet.columns = ['ds','y']
res = m.fit(prices_prophet)

In [ ]:
m = Prophet(changepoint_prior_scale=prior_scale)
m.add_country_holidays(country_name='US')
m.fit(prices_prophet)

In [ ]:
m.train_holiday_names

In [ ]:
future = m.make_future_dataframe(periods=365)
future.tail()

In [ ]:
forecast = m.predict(future)

In [ ]:
fig1 = m.plot(forecast)

In [ ]:

fig2 = m.plot_components(forecast)

In [ ]:
forecast_m = forecast[['ds','yhat']].copy()
forecast_m

# Create a new column with the format 'YYYYY-MM'.
forecast_m['YEAR_MONTH'] = forecast_m['ds'].dt.to_period('M')

forecast_m = forecast_m.groupby('YEAR_MONTH')['yhat'].mean().reset_index()
forecast_m

In [ ]:

forecast_series = forecast_m[forecast_m['YEAR_MONTH'] > monthly_avg_price['Month_Year'].iloc[-1].to_period('M')]['yhat']
forecast_series.index = forecast_index.values
print(forecast_series)


combined_dates = pd.concat([monthly_avg_price['Month_Year'], pd.Series(forecast_index)])
combined_prices = pd.concat([monthly_avg_price['Average_Price'], forecast_series])

plt.figure(figsize=(14, 6))

# Graficar datos históricos
plt.plot(combined_dates[:len(monthly_avg_price)], combined_prices[:len(monthly_avg_price)], label='Historical', color='blue')

# Graficar datos de pronóstico
plt.plot(combined_dates[len(monthly_avg_price)-1:], combined_prices[len(monthly_avg_price)-1:], label='Forecast', color='green', linestyle='--')

plt.title('Natural Gas Prices Forecast')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


### Cross Validation

In [ ]:
from prophet.diagnostics import performance_metrics
from prophet.diagnostics import cross_validation

In [ ]:
df_cv = cross_validation(m, initial='1460 days', period='180 days', horizon = '365 days')

In [ ]:
df_p = performance_metrics(df_cv)
df_p.head()

In [ ]:
from prophet.plot import plot_cross_validation_metric
fig = plot_cross_validation_metric(df_cv, metric='mape')

In [ ]:
fig = plot_cross_validation_metric(df_cv, metric='mdape')

In [ ]:
fig = plot_cross_validation_metric(df_cv, metric='rmse')